<h1>Modeling testbook</h1>

In [45]:
## Latest and greatest sklearn
# !pip install sklearn --upgrade
## Dependencies for data preprocessing
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
#import config
import json
import time
import datetime
from pprint import pprint
# from nba_api.stats.endpoints import leaguegamelog
# from nba_api.stats.endpoints import boxscoresummaryv2
import pandas
import sqlite3

In [46]:
## Connect to sqlite db
conn = sqlite3.connect("../../NBAapi/NBAapi/app.db")
## Teams cheatsheet
## Warriors NBA_Id: 1610612744, TeamId: 10
## Celtics NBA_Id: 1610612738, TeamId: 2
## Lakers NBA_Id: 1610612747, TeamId: 14
TeamName = 'Celtics'
c = conn.cursor()
c.execute("SELECT TeamId, NBA_API_TeamId FROM Team WHERE Name = '"+TeamName+"'")
## Fetching team values from the table
## Also sets all values to be used for insertion into Predicion/Summary
teamDf = c.fetchone()
TeamId = teamDf[0]
NBA_API_TeamId = teamDf[1]
Features = 6
# print(TeamId)
# print(NBA_API_TeamId)


In [47]:

## Load boxscore data into a DataFrame
## First load data from Team playing from home
home_games = "SELECT PointsHome AS Points, FGPctHome AS FGPct, FTPctHome AS FTPct, FG3PctHome AS FG3Pct, ASTHome AS AST, REBHome AS REB, HomeTeamWin AS Win "
home_games += "FROM GameBoxScore WHERE NBA_API_HomeTeamId = "+str(NBA_API_TeamId)+" ORDER BY NBA_API_GameId DESC"
df_home = pd.read_sql_query(home_games, conn)
## Then load data from away games
away_games = "SELECT PointsAway AS Points, FGPctAway AS FGPct, FTPctAway AS FTPct, FG3PctAway AS FG3Pct, ASTAway AS AST, REBAway AS REB, AwayTeamWin AS Win "
away_games += "FROM GameBoxScore WHERE NBA_API_AwayTeamId = "+str(NBA_API_TeamId)+" ORDER BY NBA_API_GameId DESC"
df_away = pd.read_sql_query(away_games, conn)
## Resetting index for both dataframes, conca/join/merge would fail without this (duplicates).
# df_home = df_home.loc[~df_home.index.duplicated(keep='first')]
# df_away = df_away.loc[~df_away.index.duplicated(keep='first')]
## Stack the DataFrames on top of each other
## This one doesn't work;; df = df_home.concat(df_away, how='left')
## Concat home and away games
df = pd.concat([df_home, df_away])
## May need to drop last column of a dataframe
# df = df.iloc[: , :-1]
## Resets dataframe index, SKlern may throw ValueError otherwise
# df = df.reset_index()
# df['Win'].to_numpy('int32')
df.head()

,Points,FGPct,FTPct,FG3Pct,AST,REB,Win
0,118,0.396,0.844,0.333,20,49,1
1,126,0.440,0.905,0.350,24,34,0
2,125,0.506,0.826,0.410,21,46,1
3,121,0.452,0.862,0.316,29,50,1
4,101,0.500,0.792,0.357,19,41,0


In [48]:
# conn.close()
# grab every stat except the 'win' column as your X features
X = df.drop('Win', axis=1)
print(X.shape)
X.head()

(228, 6)


,Points,FGPct,FTPct,FG3Pct,AST,REB
0,118,0.396,0.844,0.333,20,49
1,126,0.440,0.905,0.350,24,34
2,125,0.506,0.826,0.410,21,46
3,121,0.452,0.862,0.316,29,50
4,101,0.500,0.792,0.357,19,41


In [49]:
# Set our Wins/Ys
y = df['Win']
y.head()

0    1
1    0
2    1
3    1
4    0
Name: Win, dtype: int64

In [50]:
# Import training and scaling tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [51]:
## Split your Xs and Ys (default split 72/25)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [52]:
## Scale X data to keep value magnitude similar and mitigate possible high variance 
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

<h1>Random Forest (using sklearn)</h1>

In [53]:
# Lets try 2 different RFC with 200 and 400 estimators, and select the best from both
from sklearn.ensemble import RandomForestClassifier
# Model definition for model #1
modelRF1 = RandomForestClassifier(n_estimators=10)
# Fit our training data
modelRF1.fit(X_train_scaled, y_train)
# Model definition for model #1
modelRF2 = RandomForestClassifier(n_estimators=200)
# Fit model #2
modelRF2.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [54]:
testScore1 = modelRF1.score(X_test_scaled, y_test)
testScore2 = modelRF2.score(X_test_scaled, y_test)
# Check RFC #1 results
print(f"Model #1 Training score: {round(modelRF1.score(X_train_scaled, y_train)*100,2)}%")
print(f"Model #1 Test score: {round(testScore1*100,2)}%")
# Check RFC #2 results
print(f"Model #2 Training score: {round(modelRF2.score(X_train_scaled, y_train)*100,2)}%")
print(f"Model #2 Test score: {round(testScore2*100,2)}%")
#Select best model
if testScore1 >= testScore2:
    modelRF = modelRF1
    print('Selected model #1')
else:
    modelRF = modelRF2
    print('Selected model #2')
bestScore = modelRF.score(X_test_scaled, y_test)

Model #1 Training score: 100.0%
Model #1 Test score: 71.93%
Model #2 Training score: 100.0%
Model #2 Test score: 80.7%
Selected model #2


In [55]:
## Dump predictions into a LabelEncoder
## Compare predictions against actual game results
result_encoder = LabelEncoder()
result_encoder.fit(y_train)
encoded_test_results = modelRF.predict(X_test_scaled)
results_labels = result_encoder.inverse_transform(encoded_test_results)
print(f"7 total Predictions:   {results_labels[:7]}")
print(f"7 total Actual results: {y_test[:7].tolist()}")

7 total Predictions:   [1 1 1 1 0 1 0]
7 total Actual results: [1, 1, 1, 1, 0, 1, 1]


<h3>Random Forrest gets us to 75% accuracy predicting win/loss based on team stats per game - not bad at all!</h3>

<h1>Neural network (using keras)</h1>

In [56]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
## Convert Ys to one-hot encode
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [57]:
## Dependencies for Sequential model and Dense for the hidden layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [58]:
# Fill our model with Dense layers, using Relu activation (not used at this moment but it allows backpropagation)
# input_dim set to 6 because we have 6 X features
# Last layer set to softmax (we want a soft result, get it?) as we look for binary classification 
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=6))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [59]:
## Fit model, adam optimizer is straightforward to implement with MLnet (see roadmap)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
## Summary, very important
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               700       
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 21,102
Trainable params: 21,102
Non-trainable params: 0
_________________________________________________________________


In [61]:
## Training time, set to 100 epochs
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
171/171 - 0s - loss: 0.6820 - acc: 0.5263
Epoch 2/100
171/171 - 0s - loss: 0.6668 - acc: 0.5205
Epoch 3/100
171/171 - 0s - loss: 0.6620 - acc: 0.5205
Epoch 4/100
171/171 - 0s - loss: 0.6492 - acc: 0.5380
Epoch 5/100
171/171 - 0s - loss: 0.6402 - acc: 0.5906
Epoch 6/100
171/171 - 0s - loss: 0.6245 - acc: 0.6725
Epoch 7/100
171/171 - 0s - loss: 0.6146 - acc: 0.7018
Epoch 8/100
171/171 - 0s - loss: 0.5948 - acc: 0.7193
Epoch 9/100
171/171 - 0s - loss: 0.5913 - acc: 0.6784
Epoch 10/100
171/171 - 0s - loss: 0.5783 - acc: 0.7135
Epoch 11/100
171/171 - 0s - loss: 0.5621 - acc: 0.7602
Epoch 12/100
171/171 - 0s - loss: 0.5450 - acc: 0.7427
Epoch 13/100
171/171 - 0s - loss: 0.5348 - acc: 0.7368
Epoch 14/100
171/171 - 0s - loss: 0.5271 - acc: 0.7427
Epoch 15/100
171/171 - 0s - loss: 0.5126 - acc: 0.7544
Epoch 16/100
171/171 - 0s - loss: 0.4982 - acc: 0.7719
Epoch 17/100
171/171 - 0s - loss: 0.4894 - acc: 0.7485
Epoch 18/100
171/171 - 0s - loss: 0.5035 - acc: 0.7485
Epoch 19/100
171/17

In [62]:
## Check results (drum roll)
modelLoss, modelAccuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {modelLoss}, Accuracy: {modelAccuracy}")

57/57 - 0s - loss: 0.5392 - acc: 0.8070
Neural Network - Loss: 0.5391923126421476, Accuracy: 0.8070175647735596


In [63]:
## Compare predictions against actual game results
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [64]:
print(f"7 total Predictions:   {prediction_labels[:7]}")
print(f"7 total Actual results: {y_test[:7].tolist()}")

7 total Predictions:   [1 1 1 1 0 1 1]
7 total Actual results: [1, 1, 1, 1, 0, 1, 1]


In [65]:
## ADD PREDICTIONS TO SQLITE DB
import sqlite3
from sqlite3 import Error
import pandas
from datetime import datetime

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def insert_predictionsummary(conn, summary):
    """
    Inserts Prediction Summary
    """
    sql = ''' INSERT INTO SeriesPredictionSummary
              (DateAdded, Features, Loss, Accuracy, Recall, TrainSize, TestSize, Epochs, Technique, Comments, TeamId, NBA_API_TeamId,Team)
              VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, summary)
    conn.commit()
    return cur.lastrowid
    
def insert_predictions(conn, prediction):
    """
    Inserts Single Prediction, detail of Summary
    """
    sql = ''' INSERT INTO SinglePrediction
              (TeamId, NBA_API_TeamId, Label, Win, SeriesPredictionSummaryId)
              VALUES(?,?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, prediction)
    conn.commit()

def main():
    # database = "../../NBAapi/Persistence/app.db"
    database = "../../NBAapi/NBAapi/app.db"
    # create a database connection
    conn = create_connection(database)
    with conn:
        seriesId = insert_predictionsummary(conn, (datetime.now(), 6, modelLoss, modelAccuracy.astype(str), 0, X_train.shape[0], X_test.shape[0], 100, "CNN", "No comments", TeamId, NBA_API_TeamId,TeamName))
        for x in prediction_labels[:7]:
            insert_predictions(conn, (TeamId, NBA_API_TeamId, '', x.astype(str), seriesId))
    print(f'All data inserted to SeriesPredictionSummaryId: {seriesId}')
    conn.close()
if __name__ == '__main__':
    main()

All data inserted to SeriesPredictionSummaryId: 5
